## Benford's Law Measure Application

This part is to use the Benford's Law to annalyse whether the transaction data are fraud of each merchants or cards. We use the ratio to analyse the abnormality. Besides, we can use chi-square test to analyse it more accurately.

In [1]:
import pandas as pd
df = pd.read_excel('card transactions.xlsx')

In [2]:
df=df[df['Transtype']=='P']

In [3]:
df=df[df['Merch description'].str.contains('FEDEX')==0]

In [4]:
# df['First']=df['Amount'].astype(str).str[0].astype(int)
# extract the first digit with the Regex
df['First'] = df['Amount'].astype('str').str.extract(r'([1-9])').astype(int)

In [5]:
import numpy as np
df['FirstTwo'] = np.where((df['First']==1) |(df['First']==2), 'Y', 'N')

In [6]:
df.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,First,FirstTwo
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0,3,N
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0,1,Y
7,8,5142191182,2010-01-01,6098208200062,MIAMI COMPUTER SUPPLY,OH,45429.0,P,230.32,0,2,Y
8,9,5142258629,2010-01-01,602608969534,FISHER SCI ATL,GA,30091.0,P,62.11,0,6,N
13,14,5142124791,2010-01-01,5725000466504,CDW*GOVERNMENT INC,IL,60061.0,P,106.89,0,1,Y


In [7]:
#Calculate the Large/Small Ratio of the overall data. (The number of the Amount data start with 3-9 divided by 1-2)
all_Merchnum_count=df[['FirstTwo','Amount']].groupby('FirstTwo').agg(['count']).reset_index()

In [8]:
all_Merchnum_count

,FirstTwo,Amount
,,count
0,N,43099
1,Y,41524


In [9]:
ratio=all_Merchnum_count.loc[0][1]/all_Merchnum_count.loc[1][1]

In [10]:
ratio #Large/Small Ratio

1.0379298718813217

In [11]:
df.head()

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,First,FirstTwo
1,2,5142183973,2010-01-01,61003026333,SERVICE MERCHANDISE #81,MA,1803.0,P,31.42,0,3,N
2,3,5142131721,2010-01-01,4503082993600,OFFICE DEPOT #191,MD,20706.0,P,178.49,0,1,Y
7,8,5142191182,2010-01-01,6098208200062,MIAMI COMPUTER SUPPLY,OH,45429.0,P,230.32,0,2,Y
8,9,5142258629,2010-01-01,602608969534,FISHER SCI ATL,GA,30091.0,P,62.11,0,6,N
13,14,5142124791,2010-01-01,5725000466504,CDW*GOVERNMENT INC,IL,60061.0,P,106.89,0,1,Y


In [12]:
def bflaw(df,colname):
    #Define a function to analyse the abnormality of the first 40 transaction amount
    Cardnum_count=df.groupby([colname,'FirstTwo']).agg({'Amount':'count'})
    idx=pd.MultiIndex.from_product([Cardnum_count.index.levels[0], ['Y', 'N']])
    Cardnum_count=Cardnum_count.reindex(idx, fill_value=0).reset_index()
    Cardnum_count['Amount']=np.where(Cardnum_count['Amount']==0, 1,Cardnum_count['Amount'])
    Cardnum_count_tot=df.groupby([colname]).agg({'Amount':'count'}).reset_index()
    Cardnum_all=pd.merge(Cardnum_count, Cardnum_count_tot, left_on='level_0', right_on=colname,how='left')
    Cardnum_all['t']=(Cardnum_all['Amount_y']-15)/3
    low=Cardnum_all[Cardnum_all['level_1']=='Y']
    high=Cardnum_all[Cardnum_all['level_1']=='N']
    total=pd.merge(low, high, on='level_0',how='inner')
    #We apply this equation to do the smoothing to avoid wrongly 
    total['R']=1+(ratio*(total['Amount_x_x']/total['Amount_x_y']-1)/(1+np.exp(-total['t_x'])))
    total['1/R']=1/total['R']
    total['Rmax']=total[['R','1/R']].max(axis=1)
    totalCard=total.sort_values(by='Rmax',ascending=False)[[colname+'_x','Rmax']].head(40)
    return totalCard

In [13]:
#By Merchnum
bflaw(df,'Merchnum')

,Merchnum_x,Rmax
10740,8916500620062,2646.464746
5215,5600900060992,74.299334
9820,8078200641472,61.199914
2348,3523000628102,35.190959
12891,993620810220,32.943515
4980,55158027,27.636605
2603,3910694900001,25.289442
10662,8889817332,24.050098
10516,881145544,23.740218
8022,6844000608436,22.350921


In [14]:
df[df['Merchnum']=='8916500620062']# pretty abnormal records with repetitive amounts and no amount starts with 1 and 2.

,Recnum,Cardnum,Date,Merchnum,Merch description,Merch state,Merch zip,Transtype,Amount,Fraud,First,FirstTwo
6163,6164,5142194617,2010-01-28,8916500620062,EZPASS PREPAID TOLL,NY,10305.0,P,650.0,0,6,N
10453,10454,5142194617,2010-02-13,8916500620062,EZPASS PREPAID TOLL,NY,10305.0,P,480.0,0,4,N
12387,12388,5142194617,2010-02-20,8916500620062,EZPASS PREPAID TOLL,NY,10305.0,P,650.0,0,6,N
15809,15810,5142194617,2010-03-05,8916500620062,EZPASS PREPAID TOLL,NY,10305.0,P,650.0,0,6,N
19384,19385,5142194617,2010-03-16,8916500620062,EZPASS PREPAID TOLL,NY,10305.0,P,480.0,0,4,N
21311,21312,5142194617,2010-03-23,8916500620062,EZPASS PREPAID TOLL,NY,10305.0,P,650.0,0,6,N
25890,25891,5142194617,2010-04-08,8916500620062,EZPASS PREPAID TOLL,NY,10305.0,P,650.0,0,6,N
26354,26355,5142194617,2010-04-11,8916500620062,EZPASS PREPAID TOLL,NY,10305.0,P,480.0,0,4,N
29729,29730,5142194617,2010-04-24,8916500620062,EZPASS PREPAID TOLL,NY,10305.0,P,650.0,0,6,N
34722,34723,5142194617,2010-05-10,8916500620062,EZPASS PREPAID TOLL,NY,10305.0,P,650.0,0,6,N


In [15]:
#By Cardnum
bflaw(df,'Cardnum')

,Cardnum_x,Rmax
725,5142197563,12.778523
1177,5142253356,12.624814
1541,5142299705,8.512889
697,5142194617,8.351125
284,5142144931,5.868111
1061,5142239140,4.559438
648,5142189113,4.409898
942,5142224699,3.917470
1639,5142847398,3.866151
680,5142192606,3.854307
